In [1]:
def construction_matrices(nodes):
    """
    Cette fonction prend en entrée la liste des noeuds qui ont été prédéfinis. 
    Elle renvoie d'une part la matrice des itinéraires entre les noeuds et d'autre part la matrice des temps de trajet entre les noeuds. 
    """
    N_nodes = len(nodes)
    matrix_durations = np.zeros((N_nodes,N_nodes))
    liste_itineraires = []
    i = 0 
    for node_start in nodes : 
        liste_itineraires.append([])
        start_x = nodes[node_start][1] 
        start_y = nodes[node_start][0]
        print(f'i = {i}')
        j = 0
        for node_end in nodes :
            end_x = nodes[node_end][1] 
            end_y = nodes[node_end][0]
            texte = requests.get("https://wxs.ign.fr/essentiels/geoportail/itineraire/rest/1.0.0/route?resource=bdtopo-osrm&start=" + f'{str(start_x)},{str(start_y)}&end={str(end_x)},{str(end_y)}&timeUnit=second')
            texte = json.loads(texte.text)
            itineraire = texte['geometry']['coordinates']
            duration = texte['duration']
            matrix_durations[i,j] = duration 
            for element in itineraire: 
                if round(element[0]) == 2 : 
                    p = element[0]
                    element[0] = element[1]
                    element[1] = p 
            itineraire_points = change(itineraire,  0.00025, 0.0002, 0.000225)
            #il va maintenant falloir modifier cela en liste de noeuds. 
            itineraire_noeuds = []
            for point in itineraire_points :
                for node in nodes : 
                    if node not in itineraire_noeuds and dist(point, nodes[node]) <= 0.00045 : 
                        itineraire_noeuds.append(node)
            liste_itineraires[i].append(itineraire_noeuds) #Donc liste_itineraires[i][j] c'est l'itineraire pour aller de i vers j en parlant en noeud
            j+=1
        i+=1 
    return liste_itineraires, matrix_durations